In [2]:
import torch, math
import pandas as pd
from IPython.display import display

torch.set_default_dtype(torch.float64)

# ===== Utilitários de derivação =====
def grad_scalar(f, x):
    x_ = x.clone().detach().requires_grad_(True)
    y = f(x_)
    if y.ndim != 0:
        raise ValueError('f(x) deve retornar um escalar para gradiente.')
    g, = torch.autograd.grad(y, x_, create_graph=True)
    return y, g, x_

def hessian_scalar(f, x):
    y, g, x_ = grad_scalar(f, x)
    rows = []
    for i in range(x_.numel()):
        gi, = torch.autograd.grad(g[i], x_, retain_graph=True, create_graph=True)
        rows.append(gi)
    H = torch.stack(rows)
    return y, g, H


def jacobian_vector(F, x):
    x_ = x.clone().detach().requires_grad_(True)
    y = F(x_)
    if y.ndim != 1:
        raise ValueError('F(x) deve retornar um vetor 1D para Jacobiano.')
    rows = []
    for i in range(y.numel()):
        Ji, = torch.autograd.grad(y[i], x_, retain_graph=True, create_graph=True)
        rows.append(Ji)
    J = torch.stack(rows)  # (m, n) com m = componentes de F, n = variáveis
    return y, J, x_

def divergence(F, x):
    y, J, _ = jacobian_vector(F, x)
    if J.shape[0] != x.numel():
        raise ValueError('Para divergente, F deve ter o mesmo nº de componentes de x.')
    return torch.trace(J)


def curl3(F, x):
    # Rotacional em R^3
    if x.numel() != 3:
        raise ValueError('curl3 exige x em R^3.')
    y, J, _ = jacobian_vector(F, x)  # J[i,j] = dF_i/dx_j
    if y.numel() != 3:
        raise ValueError('curl3 exige F: R^3 -> R^3.')
    return torch.stack([
        J[2,1] - J[1,2],
        J[0,2] - J[2,0],
        J[1,0] - J[0,1]
    ])

def curl2(F, x):
    # Versão 2D (componente k): ∂Fy/∂x - ∂Fx/∂y
    if x.numel() != 2:
        raise ValueError('curl2 exige x em R^2.')
    y, J, _ = jacobian_vector(F, x)
    if y.numel() != 2:
        raise ValueError('curl2 exige F: R^2 -> R^2.')
    return J[1,0] - J[0,1]

# ===== Utilitários de exibição =====
def df_vec(v, names, title=None):
    df = pd.DataFrame({"nome": names, "valor": [float(vi) for vi in v.detach()]})
    if title: print(title)
    display(df)

def df_mat(M, row_names=None, col_names=None, title=None):
    df = pd.DataFrame(M.detach().numpy(), index=row_names, columns=col_names)
    if title: print(title)
    display(df)

In [3]:
names = ['x','y','z']
def f(u):
    x, y, z = u
    return x**2 * y + torch.sin(z) + y*z**2 - 3*x

p = torch.tensor([1.0, 2.0, 0.5])
val, g, H = hessian_scalar(f, p)
print('f(p) =', float(val))
df_vec(g, names, 'Gradiente ∇f')
df_mat(H, names, names, 'Hessiana ∇²f')

f(p) = -0.020574461395796995
Gradiente ∇f


,nome,valor
0,x,1.000000
1,y,1.250000
2,z,2.877583


Hessiana ∇²f


,x,y,z
x,4.0,2.0,0.000000
y,2.0,0.0,1.000000
z,0.0,1.0,3.520574


In [4]:
names3 = ['x','y','z']
comp_names = ['F1','F2','F3']
def F(u):
    x, y, z = u
    return torch.stack([x*y, y*z, torch.sin(x) + z**2])

q = torch.tensor([1.0, 2.0, 3.0])
Fval, J, _ = jacobian_vector(F, q)
divF = divergence(F, q)
curlF = curl3(F, q)
df_vec(Fval, comp_names, 'F(q)')
df_mat(J, comp_names, names3, 'Jacobiano J_F')
print('div F(q) =', float(divF))
df_vec(curlF, ['i','j','k'], 'Rotacional ∇×F (3D)')

F(q)


,nome,valor
0,F1,2.000000
1,F2,6.000000
2,F3,9.841471


Jacobiano J_F


,x,y,z
F1,2.000000,1.0,0.0
F2,0.000000,3.0,2.0
F3,0.540302,0.0,6.0


div F(q) = 11.0
Rotacional ∇×F (3D)


,nome,valor
0,i,-2.000000
1,j,-0.540302
2,k,-1.000000


In [5]:
names2 = ['x','y']
comp2 = ['G1','G2']
def G(u):
    x, y = u
    return torch.stack([x**2 - y, x*y + torch.cos(x)])

r = torch.tensor([math.pi/4, -1.0])
Gval, Jg, _ = jacobian_vector(G, r)
divG = divergence(G, r)
curlG = curl2(G, r)
df_vec(Gval, comp2, 'G(r)')
df_mat(Jg, comp2, names2, 'Jacobiano J_G')
print('div G(r) =', float(divG))
print('curl (2D) = ∂G_y/∂x - ∂G_x/∂y =', float(curlG))

G(r)


,nome,valor
0,G1,1.616850
1,G2,-0.078291


Jacobiano J_G


,x,y
G1,1.570796,-1.000000
G2,-1.707107,0.785398


div G(r) = 2.356194490192345
curl (2D) = ∂G_y/∂x - ∂G_x/∂y = -0.7071067811865475
